In [1]:
import pandas as pd

In [2]:
debt = pd.read_csv('debt.csv')

In [3]:
debt.rename(columns={'2019 [YR2019]': '2019', '2020 [YR2020]': '2020', '2021 [YR2021]': '2021', '2022 [YR2022]': '2022', '2023 [YR2023]': '2023'}, inplace=True)

In [4]:
debt.dropna(inplace=True)

In [5]:
debt['Country Name'] = debt['Country Name'].str.strip()
debt['Counterpart-Area Name'] = debt['Counterpart-Area Name'].str.strip()

In [6]:
debt = debt[(debt['2019'] != '..') & (debt['2020'] != '..') & (debt['2021'] != '..') & (debt['2022'] != '..') & (debt['2023'] != '..')]

In [7]:
debt = debt.melt(id_vars=['Country Name', 'Country Code', 'Counterpart-Area Name'], value_vars=['2019', '2020', '2021', '2022', '2023'], var_name='Year', value_name='Debt')

In [8]:
debt['Debt'] = debt['Debt'].astype(float)

In [9]:
print('Hello')

Hello


In [10]:
pd.DataFrame(debt.groupby('Counterpart-Area Name')['Debt'].sum().sort_values(ascending=False))

,Debt
Counterpart-Area Name,
World,4.316449e+11
Other Multiple Lenders,1.671310e+11
China,6.572703e+10
Bondholders,4.959060e+10
World Bank-IDA,2.081378e+10
...,...
Grenada,4.588891e+05
St. Lucia,3.034261e+05
International Bank for Economic Cooperation (IBEC),1.855000e+05


In [16]:
grouped_2019 = debt[debt['Year'] == "2019"].groupby(['Counterpart-Area Name', 'Country Name'])[['Debt']].sum()
grouped_2020 = debt[debt['Year'] == "2020"].groupby(['Counterpart-Area Name', 'Country Name'])[['Debt']].sum()
grouped_2021 = debt[debt['Year'] == "2021"].groupby(['Counterpart-Area Name', 'Country Name'])[['Debt']].sum()
grouped_2022 = debt[debt['Year'] == "2022"].groupby(['Counterpart-Area Name', 'Country Name'])[['Debt']].sum()

import json

def save_json(grouped, year):
    levels = len(grouped.index.levels)
    dicts = [{} for i in range(levels)]
    last_index = None

    for index,value in grouped.itertuples():

        if not last_index:
            last_index = index

        for (ii,(i,j)) in enumerate(zip(index, last_index)):
            if not i == j:
                ii = levels - ii -1
                dicts[:ii] =  [{} for _ in dicts[:ii]]
                break

        for i, key in enumerate(reversed(index)):
            dicts[i][key] = value
            value = dicts[i]

        last_index = index


    result = dicts[-1]
    # result = json.dumps(dicts[-1])

    debt_restructured = {'name': 'debt', 'children': []}

    for i in result:
        obj = {'name': i, 'children': []}
        for j in result[i]:
            obj['children'].append({'name': j, 'value': result[i][j]})
        debt_restructured['children'].append(obj)
    print(debt_restructured)

    with open(f'debt_{year}.json', 'w') as f:
        f.write(json.dumps(debt_restructured, indent=4))
        
save_json(grouped_2019, 2019)
save_json(grouped_2020, 2020)
save_json(grouped_2021, 2021)
save_json(grouped_2022, 2022)

{'name': 'debt', 'children': [{'name': 'African Dev. Bank', 'children': [{'name': 'Angola', 'value': 72764968.4}, {'name': 'Benin', 'value': 8051068.1}, {'name': 'Burkina Faso', 'value': 10043455.7}, {'name': 'Burundi', 'value': 735055.3}, {'name': 'Cabo Verde', 'value': 13559839.1}, {'name': 'Cameroon', 'value': 14987118.3}, {'name': 'Central African Republic', 'value': 38687.2}, {'name': 'Chad', 'value': 2662503.4}, {'name': 'Comoros', 'value': 0.0}, {'name': 'Congo, Dem. Rep.', 'value': 118837197.9}, {'name': 'Congo, Rep.', 'value': 826246.7}, {'name': "Cote d'Ivoire", 'value': 5253160.0}, {'name': 'Djibouti', 'value': 4241767.7}, {'name': 'Ethiopia', 'value': 44465331.3}, {'name': 'Gambia, The', 'value': 2086341.7}, {'name': 'Ghana', 'value': 21074815.0}, {'name': 'Guinea', 'value': 3205505.0}, {'name': 'Guinea-Bissau', 'value': 788941.1}, {'name': 'Kenya', 'value': 41197650.0}, {'name': 'Lesotho', 'value': 7133630.9}, {'name': 'Liberia', 'value': 1388591.9}, {'name': 'Madagascar',

In [12]:
dicts[-1]

NameError: name 'dicts' is not defined

In [17]:
debt.to_csv('debt_clean.csv', index=False)

In [13]:
grouped_2019

,,Debt
Counterpart-Area Name,Country Name,
